### Логистическая регрессия и полезные преобразования

Данные о доходах населения.  
Источник данных: https://www.kaggle.com/uciml/adult-census-income

In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('adult.csv') # Загружаем данные
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country,class
0,2,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,1,0,2,United-States,<=50K
1,3,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,0,United-States,<=50K
2,2,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,2,United-States,<=50K
3,3,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,2,United-States,<=50K
4,1,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,2,Cuba,<=50K


In [4]:
# Заменим ? на NaN
df = df.replace('?',np.nan)

In [4]:
# Узнаем где есть пропуски
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  31978 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [5]:
# Удалим строки с пропусками
df = df.dropna()

In [7]:
df['class'].value_counts()

<=50K    34014
>50K     11208
Name: class, dtype: int64

#### Признаки

Когда мы говорим о категориальных данных, мы должны различать **номинальные** и **порядковые признаки**.              
***Порядковые признаки*** могут пониматься как категориальные значения, которые могут быть отсортированы или упорядочены.   
***Hоминальные признаки*** не подразумевают порядка.

Нужно помнить, что метки классов не являются порядковыми, и поэтому не имеет значения, какое целое число мы присваиваем отдельно взятой строковой метке.

In [8]:
# Какой это признак?
df['education'].value_counts()

HS-grad         14783
Some-college     9899
Bachelors        7570
Masters          2514
Assoc-voc        1959
11th             1619
Assoc-acdm       1507
10th             1223
7th-8th           823
Prof-school       785
9th               676
12th              577
Doctorate         544
5th-6th           449
1st-4th           222
Preschool          72
Name: education, dtype: int64

In [10]:
df['education-num'].value_counts()

9     14783
10     9899
13     7570
14     2514
11     1959
7      1619
12     1507
6      1223
4       823
15      785
5       676
8       577
16      544
3       449
2       222
1        72
Name: education-num, dtype: int64

In [11]:
del df['education']

In [12]:
y = df['class'].replace({'<=50K': 0, '>50K': 1})   # Ответы 

In [14]:
# Перекодируем в бинарные признаки
df['sex'] = df['sex'].replace({'Female': 0, 'Male': 1})
df['race'] = df['race'].replace({'White': 0, 'Black': 1})

X = df.drop('class', axis=1)                      # Признаки

In [15]:
X

,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country
0,2,State-gov,77516,13,Never-married,Adm-clerical,Not-in-family,0,1,1,0,2,United-States
1,3,Self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,0,1,0,0,0,United-States
2,2,Private,215646,9,Divorced,Handlers-cleaners,Not-in-family,0,1,0,0,2,United-States
3,3,Private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,1,1,0,0,2,United-States
4,1,Private,338409,13,Married-civ-spouse,Prof-specialty,Wife,1,0,0,0,2,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48836,1,Private,245211,13,Never-married,Prof-specialty,Own-child,0,1,0,0,2,United-States
48837,2,Private,215419,13,Divorced,Prof-specialty,Not-in-family,0,0,0,0,2,United-States
48839,2,Private,374983,13,Married-civ-spouse,Prof-specialty,Husband,0,1,0,0,3,United-States
48840,2,Private,83891,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,1,2,0,2,United-States


#### Рассмотрим пример
Пусть у нас есть данные, в которых есть **признак** "Цвет". В столбце 3 значения: синий, зеленый, красный. Кодируем:

синий —> 0  
зеленый —> 1   
красный —> 2  


Сейчас мы сделали одну из наиболее распространенных ошибок, которая встречается во время 
работы с категориальными данными. Можете определить, в чем проблема?

Несмотря на то, что значения цвета не идут в том или ином конкретном порядке, алгоритм обучения предположит, что зеленый больше синего, а красный больше зеленого. Хотя это допущение является неправильным, алгоритм все равно способен произвести полезные результаты. Однако эти результаты не будут оптимальными.

#### Прямое кодирование (One Hot Encoding)

![img](https://miro.medium.com/max/1200/1*ggtP4a5YaRx6l09KQaYOnw.png)

Можно использовать ColumnTransformer из sklearn.compose и OneHotEncoder из sklearn.preprocessing:  
https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [16]:
# А можно еще проще: 
X_transform = pd.get_dummies(X)
X_transform

,age,fnlwgt,education-num,sex,capitalgain,capitalloss,hoursperweek,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,2,77516,13,1,1,0,2,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,83311,13,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2,215646,9,1,0,0,2,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,3,234721,7,1,0,0,2,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,1,338409,13,0,0,0,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48836,1,245211,13,1,0,0,2,0,0,1,...,0,0,0,0,0,0,0,1,0,0
48837,2,215419,13,0,0,0,2,0,0,1,...,0,0,0,0,0,0,0,1,0,0
48839,2,374983,13,1,0,0,3,0,0,1,...,0,0,0,0,0,0,0,1,0,0
48840,2,83891,13,1,2,0,2,0,0,1,...,0,0,0,0,0,0,0,1,0,0


Большинство алгоритмов машинного обучения ведут себя гораздо лучше, если все признаки измеряются по одной шкале.  
Существуют два общих подхода к приведению разных признаков к одной шкале: нормализация и стандартизация. В различных областях эти термины нередко используются довольно нечетко, и их конкретное содержание приходится выводить из контекста. Чаще всего нормализация означает приведение (нормирование) признаков к диапазону [0, 1] (минимаксное масштабирование, https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html).   
Несмотря на то, что широко используемый метод нормализация путем минимаксного масштабирования целесообразно использовать, когда нам нужны значения в ограниченном интервале, для многих алгоритмов машинного обучения может быть более практичной cтандартизация. Стандартизация набора данных подразумевает такое масштабирование данных, при котором каждый признак имеет среднее значение равное нулю и дисперсию равную единице. Многие линейные модели, такие как логистическая регрессия и метод опорных векторов инициализируют веса нулями либо малыми случайными величинами, близкими к 0. При помощи стандартизации столбцы имеют нормальное распределение, что упрощает извлечение весов. Кроме того, стандартизация содержит полезную информацию о выбросах и делает алгоритм менее к ним чувствительным.

In [17]:
from sklearn.preprocessing import StandardScaler 

# Осуществим стандартизацию
scale = StandardScaler() # Создаем экземпляр класса
X_scale = scale.fit_transform(X_transform) # Преобразуем данные

In [18]:
X_scale

array([[ 0.18192956, -1.06229487,  1.12875281, ...,  0.30850579,
        -0.04288082, -0.02255794],
       [ 0.97133836, -1.00743773,  1.12875281, ...,  0.30850579,
        -0.04288082, -0.02255794],
       [ 0.18192956,  0.24528351, -0.43812161, ...,  0.30850579,
        -0.04288082, -0.02255794],
       ...,
       [ 0.18192956,  1.75361345,  1.12875281, ...,  0.30850579,
        -0.04288082, -0.02255794],
       [ 0.18192956, -1.00194728,  1.12875281, ...,  0.30850579,
        -0.04288082, -0.02255794],
       [-0.60747924, -0.07181821,  1.12875281, ...,  0.30850579,
        -0.04288082, -0.02255794]])

In [19]:
y

0        0
1        0
2        0
3        0
4        0
        ..
48836    0
48837    0
48839    0
48840    0
48841    1
Name: class, Length: 45222, dtype: int64

In [20]:
from sklearn.model_selection import train_test_split

# Разбиваем выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size = 0.3, random_state = 42)

In [21]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import classification_report # Позволяет получить сразу несколько метрик

lr = LogisticRegression() # Создаем экземпляр класса
lr.fit(X_train, y_train)  # Обучаем
y_pred = lr.predict(X_test) # Делаем предсказание на тестовой выборке

print(classification_report(y_test, y_pred)) # Оценим качество

              precision    recall  f1-score   support

           0       0.88      0.93      0.90     10193
           1       0.75      0.61      0.67      3374

    accuracy                           0.85     13567
   macro avg       0.81      0.77      0.79     13567
weighted avg       0.84      0.85      0.85     13567



In [22]:
lr.predict_proba(X_test) # Вероятности принадлежности классам

array([[0.25374385, 0.74625615],
       [0.93047293, 0.06952707],
       [0.88922135, 0.11077865],
       ...,
       [0.72086475, 0.27913525],
       [0.67655347, 0.32344653],
       [0.69886666, 0.30113334]])

In [23]:
lr.predict(X_test)

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)